In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from citipy import citipy
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [6]:
# Import WeatherPy data into a dataframe
citiesfile = "../WeatherPy/output_data/cities.csv"
citiesdf = pd.read_csv(citiesfile)
citiesdf

,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,7.8000,-8.5333,79.25,18,9,2.06,GN,1643569803
1,-9.8000,-139.0333,77.81,77,26,14.65,PF,1643569803
2,11.6667,92.7500,72.54,71,9,7.14,IN,1643569804
3,73.5069,80.5464,-2.52,93,100,21.36,RU,1643569804
4,27.9769,-114.0611,64.92,55,97,13.78,MX,1643569776
...,...,...,...,...,...,...,...,...
565,44.0787,-80.2041,21.67,68,97,8.72,CA,1643570669
566,-37.8811,147.9810,66.58,98,98,2.66,AU,1643570669
567,68.0531,39.5131,2.25,98,99,7.85,RU,1643570669
568,-2.6500,10.0000,78.12,84,100,3.98,GA,1643570670


In [4]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Create a marker_layer using the poverty list to fill the info box
locations = citiesdf[["Lat", "Lon"]]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= citiesdf["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Narrow down the cities to meet the following criteria:
#  * max temperature between 75-85 degrees
#  * wind speed below 10 mph
#  * zero cloudiness

# Identify cities not meeting the criteria above
notidealcities = citiesdf[(citiesdf["Cloudiness"] > 0) | (citiesdf["Wind Speed"] >= 10) | (citiesdf["Max Temp"] > 85) | (citiesdf["Max Temp"] < 75)].index

# Drop the not ideal cities from our list
bestweather = citiesdf.drop(notidealcities)
bestweather.reset_index()

,index,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,211,20.8947,-156.4700,77.85,60,0,5.75,US,1643570117
1,253,-13.7000,-76.2167,75.25,64,0,6.91,PE,1643570192
2,303,34.4133,-119.8610,75.69,41,0,6.91,US,1643570272
3,336,-21.2167,48.3333,75.06,75,0,1.83,MG,1643570284
4,450,17.9583,-102.2000,83.25,52,0,7.52,MX,1643570505
5,491,-22.9000,44.5333,77.76,51,0,7.58,MG,1643570520


In [49]:
# Oops! We didn't put city names in our WeatherPy dataframe!

finalcities = {}

lats = bestweather["Lat"]
lngs = bestweather["Lon"]
lat_lngs = zip(lats, lngs)

for lat_lng in lat_lngs:
    finalcities[lat_lng] = {"City Name": (citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name),
                  "Lat": lat_lng[0],
                  "Lon": lat_lng[1]
                  }

finalcitiesdf = pd.DataFrame.from_dict(finalcities, orient='index')
finalcitiesdf.reset_index()
bestweather = bestweather.merge(finalcitiesdf, on="Lat")
del bestweather["Lon_y"]
bestweather.rename(columns={"Lon_x": "Lon"}, inplace=True)
bestweather

,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City Name
0,20.8947,-156.4700,77.85,60,0,5.75,US,1643570117,kahului
1,-13.7000,-76.2167,75.25,64,0,6.91,PE,1643570192,pisco
2,34.4133,-119.8610,75.69,41,0,6.91,US,1643570272,isla vista
3,-21.2167,48.3333,75.06,75,0,1.83,MG,1643570284,mananjary
4,17.9583,-102.2000,83.25,52,0,7.52,MX,1643570505,lazaro cardenas
5,-22.9000,44.5333,77.76,51,0,7.58,MG,1643570520,sakaraha


In [80]:
# Create a dataframe of hotels in the cities with great weather

bestweatherhotels = []

# geocoordinates
cities = ['20.8947,-156.4700',
          '-13.7000,-76.2167',
          '34.4133,-119.8610',
          '-21.2167,48.3333',
          '17.9583,-102.2000',
         '-22.9000,44.5333']

for city in cities:
    # target_coordinates = [bestweather["Lat", "Lon"]]
    target_coordinates = city
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    hotels = response.json()
    # add the hotel to the dictionary
    bestweatherhotels.append(hotels['results'][0]['name'])
    
bestweatherhotels

['Maui Seaside Hotel',
 'Hotel San Jorge Residencial',
 'Hilton Garden Inn Santa Barbara/Goleta',
 'Sorafa Hotel',
 'Hotel de Casablanca',
 'Palace Club']

In [81]:
finaldata = bestweather.copy()

finaldata['Hotel'] = bestweatherhotels
finaldata

,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City Name,Hotel
0,20.8947,-156.4700,77.85,60,0,5.75,US,1643570117,kahului,Maui Seaside Hotel
1,-13.7000,-76.2167,75.25,64,0,6.91,PE,1643570192,pisco,Hotel San Jorge Residencial
2,34.4133,-119.8610,75.69,41,0,6.91,US,1643570272,isla vista,Hilton Garden Inn Santa Barbara/Goleta
3,-21.2167,48.3333,75.06,75,0,1.83,MG,1643570284,mananjary,Sorafa Hotel
4,17.9583,-102.2000,83.25,52,0,7.52,MX,1643570505,lazaro cardenas,Hotel de Casablanca
5,-22.9000,44.5333,77.76,51,0,7.58,MG,1643570520,sakaraha,Palace Club


In [91]:
# Configure gmaps
gmaps.configure(api_key=g_key)
hotels = finaldata['Hotel'].tolist()
finalcities = finaldata['City Name'].tolist()

# Create a marker_layer using the poverty list to fill the info box
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights= finaldata["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in finaldata.iterrows()]
locations = finaldata[["Lat", "Lon"]]

markers = gmaps.marker_layer(locations,
    info_box_content=hotel_info)

fig.add_layer(markers)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))